In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cuda:2"

In [3]:
from sig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
)

In [4]:
output_dir = "talklife_moc_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

TalkLife MoC

In [5]:
%run load_talklifemoc.py

In [6]:
%run load_sbert-embeddings.py

In [7]:
sbert_embeddings.shape

torch.Size([18604, 384])

SWNU Network

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
include_features_in_path = True
include_features_in_input = False

In [9]:
# create indices for kfold
fold_col_names = [c for c in df.columns if "fold" in c]
fold_list = []
for foldc in fold_col_names:
    fold_list.append(
        (
            df[df[foldc] == "train"].index,
            df[df[foldc] == "dev"].index,
            df[df[foldc] == "test"].index,
        )
    )
fold_list = tuple(fold_list)

In [10]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]
dimreduction_method = ["umap"]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [11]:
size = 5
(
    swnu_network_umap_5,
    best_swnu_network_umap_5,
    _,
    __,
) = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=dimreduction_method,
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=None,
    split_indices=fold_list,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/18604 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in talklife_moc_output/swnu_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in talklife_moc_output/swnu_network_umap_focal_2_5_kfold_best_model.csv


In [12]:
swnu_network_umap_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_206768/1481864875.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.805884   
                                                       0.0005         0.799828   
                                                       0.0010         0.805830   
                                          0.2          0.0001         0.806314   
                                                       0.0005         0.795205   
                                                       0.0010         0.804970   
                           (512, 512)     0.1          0.0001         0.805275   
                                                       0.0005         0.803429   
                                                       0.0010         0.797284   
                                          0.2          0.0001         0.804146   
                                                       0.0005         0.801208   
                                                       0.0010         0.802731   
           (12,)           (256, 256)     0.1          0.0001         0.809933   
                                                       0.0005         0.806457   
                                                       0.0010         0.801638   
                                          0.2          0.0001         0.812155   
                                                       0.0005         0.806368   
                                                       0.0010         0.802659   
                           (512, 512)     0.1          0.0001         0.806242   
                                                       0.0005         0.799488   
                                                       0.0010         0.799165   
                                          0.2          0.0001         0.807801   
                                                       0.0005         0.805024   
                                                       0.0010         0.797785   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.545227   
                                                       0.0005         0.546562   
                                                       0.0010         0.546926   
                                          0.2          0.0001         0.546321   
                                                       0.0005         0.546315   
                                                       0.0010         0.549334   
                           (512, 512)     0.1          0.0001         0.543577   
                                                       0.0005         0.547373   
                                                       0.0010         0.544266   
                                          0.2          0.0001         0.544270   
                                                       0.0005         0.545864   
                                                       0.0010         0.543561   
           (12,)           (256, 256)     0.1          0.0001         0.551504   
                                                       0.0005         0.550656   
                                                       0.0010         0.548553   
                                          0.2          0.0001         0.552037   
                                                       0.0005         0.549335   
                                                       0.0010         0.548784   
                           (512, 512)     0.1          0.0001         0.548164   
                                                       0.0005         0.548543   
                                                       0.0010         0.545235   
                                          0.2      

In [13]:
best_swnu_network_umap_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.807568,0.549454,"[0.887579710614055, 0.4675499663148439, 0.2932...",0.536311,"[0.9026269010468102, 0.42716454657365616, 0.27...",0.566073,"[0.8730259806418746, 0.5163690476190477, 0.308...",None,0.804218,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.820092,0.555453,"[0.8965517241379309, 0.46545974273463553, 0.30...",0.552128,"[0.9007584522432189, 0.4477543538038497, 0.307...",0.559304,"[0.8923841059602649, 0.48462301587301587, 0.30...",None,0.814639,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.808805,0.551204,"[0.8886371727240364, 0.4646419862558191, 0.300...",0.539655,"[0.9035208950312603, 0.42004008016032063, 0.29...",0.566502,"[0.8742358634742741, 0.5198412698412699, 0.305...",None,0.803510,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [20]:
best_swnu_network_umap_5[
    [
        "f1",
        "f1_scores",
        "precision",
        "recall",
        "valid_f1",
        "valid_f1_scores",
        "valid_precision",
        "valid_recall",
        "dimensions",
        "swnu_hidden_dim",
        "sig_depth",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
        "seed",
        "loss_function",
        "k_fold",
        "batch_size",
    ]
]

,f1,f1_scores,precision,recall,valid_f1,valid_f1_scores,valid_precision,valid_recall,dimensions,swnu_hidden_dim,sig_depth,ffn_hidden_dim,dropout_rate,learning_rate,seed,loss_function,k_fold,batch_size
0,0.549454,"[0.887579710614055, 0.4675499663148439, 0.2932...",0.536311,0.566073,0.550683,"[0.8851225056832535, 0.47760188615695526, 0.28...",0.536954,0.569600,15,"(12,)",3,"(256, 256)",0.2,0.0001,1,focal,True,64
0,0.555453,"[0.8965517241379309, 0.46545974273463553, 0.30...",0.552128,0.559304,0.553083,"[0.8925417198328705, 0.4839670579989487, 0.282...",0.546934,0.562002,15,"(12,)",3,"(256, 256)",0.2,0.0001,12,focal,True,64
0,0.551204,"[0.8886371727240364, 0.4646419862558191, 0.300...",0.539655,0.566502,0.551075,"[0.8844772646805927, 0.47520184544405997, 0.29...",0.538718,0.570158,15,"(12,)",3,"(256, 256)",0.2,0.0001,123,focal,True,64


In [14]:
best_swnu_network_umap_5["f1"].mean()

0.5520372128545254

In [15]:
best_swnu_network_umap_5["precision"].mean()

0.5426979463920379

In [16]:
best_swnu_network_umap_5["recall"].mean()

0.5639597393833856

In [17]:
np.stack(best_swnu_network_umap_5["f1_scores"]).mean(axis=0)

array([0.89092287, 0.4658839 , 0.29930487])

In [18]:
np.stack(best_swnu_network_umap_5["precision_scores"]).mean(axis=0)

array([0.90230208, 0.43165299, 0.29413876])

In [19]:
np.stack(best_swnu_network_umap_5["recall_scores"]).mean(axis=0)

array([0.87988198, 0.50694444, 0.30505279])